In [1]:
import requests
import json
import pandas as pd 
from importlib import reload 
import gzip 
import connection


connection = reload(connection) 


<connection object at 0x784c3e1e9940; dsn: 'user=admin password=xxx dbname=dev host=coinbase-cluster.ctpfzvbnyqdb.us-east-1.redshift.amazonaws.com port=5439', closed: 0>
Connection to Postgres Server: coinbase-cluster.ctpfzvbnyqdb.us-east-1.redshift.amazonaws.com is successful on Port: 5439. Database is dev
Connection to Postgres Server: 172.17.0.2 is successful on Port: 5432. Database is coinbase10
<connection object at 0x784c3cac2d40; dsn: 'user=admin password=xxx dbname=dev host=coinbase-cluster.ctpfzvbnyqdb.us-east-1.redshift.amazonaws.com port=5439', closed: 0>
Connection to Postgres Server: coinbase-cluster.ctpfzvbnyqdb.us-east-1.redshift.amazonaws.com is successful on Port: 5439. Database is dev
Connection to Postgres Server: 172.17.0.2 is successful on Port: 5432. Database is coinbase10


In [2]:
# create connection object postgres db 
local_con =connection.local_pgcon
local_cur = connection.local_pgcur

print(local_con) 
print("\n")
print(local_cur) 


<connection object at 0x784c3cac2ac0; dsn: 'user=postgres password=xxx dbname=coinbase10 host=172.17.0.2 port=5432', closed: 0>


<built-in method cursor of psycopg2.extensions.connection object at 0x784c3cac2ac0>


In [3]:
# create s3 object
s3 = connection.s3_bucket

# create Redshift connection and cursor
cur_redshift = connection.redshift_cur
conn_redshift = connection.redshift_con
# print(cur_redshift)
print(cur_redshift)

#create local postgres connection and cursor
cur_pg = connection.local_pgcur
conn_pg = connection.local_pgcon



<built-in method cursor of psycopg2.extensions.connection object at 0x784c3e1e9940>


In [4]:
# delete this block
# conn_redshift = connection.conn_redshift
# cur_redshift = connection.cur_redshift
# print(cur_redshift)

In [28]:
url2 ='https://api.binance.com/api/v3/ticker/price'

url = 'https://api.coinbase.com/v2/currencies'
response = requests.get(url2)         

In [47]:
# a=response.text
# ajson = json.loads(a)
df = pd.DataFrame(ajson)
df

,symbol,price
0,ETHBTC,0.04664000
1,LTCBTC,0.00125200
2,BNBBTC,0.00864100
3,NEOBTC,0.00023220
4,QTUMETH,0.00119500
...,...,...
2635,ARKMUSDC,2.40340000
2636,ARUSDC,46.84300000
2637,BBUSDC,0.35480000
2638,CRVUSDC,0.43460000


In [34]:
# api_data = response.text
#api_data 

list_data = json.loads(api_data)
list_data

[{'symbol': 'ETHBTC', 'price': '0.04664000'},
 {'symbol': 'LTCBTC', 'price': '0.00125200'},
 {'symbol': 'BNBBTC', 'price': '0.00864100'},
 {'symbol': 'NEOBTC', 'price': '0.00023220'},
 {'symbol': 'QTUMETH', 'price': '0.00119500'},
 {'symbol': 'EOSETH', 'price': '0.00026060'},
 {'symbol': 'SNTETH', 'price': '0.00001190'},
 {'symbol': 'BNTETH', 'price': '0.00023630'},
 {'symbol': 'BCCBTC', 'price': '0.07908100'},
 {'symbol': 'GASBTC', 'price': '0.00007730'},
 {'symbol': 'BNBETH', 'price': '0.18530000'},
 {'symbol': 'BTCUSDT', 'price': '66930.70000000'},
 {'symbol': 'ETHUSDT', 'price': '3120.65000000'},
 {'symbol': 'HSRBTC', 'price': '0.00041400'},
 {'symbol': 'OAXETH', 'price': '0.00017780'},
 {'symbol': 'DNTETH', 'price': '0.00002801'},
 {'symbol': 'MCOETH', 'price': '0.00577200'},
 {'symbol': 'ICNETH', 'price': '0.00166300'},
 {'symbol': 'MCOBTC', 'price': '0.00021140'},
 {'symbol': 'WTCBTC', 'price': '0.00000024'},
 {'symbol': 'WTCETH', 'price': '0.00023700'},
 {'symbol': 'LRCBTC', 'p

In [37]:
df = pd.DataFrame(list_data)
df

,symbol,price
0,ETHBTC,0.04664000
1,LTCBTC,0.00125200
2,BNBBTC,0.00864100
3,NEOBTC,0.00023220
4,QTUMETH,0.00119500
...,...,...
2635,ARKMUSDC,2.40340000
2636,ARUSDC,46.84300000
2637,BBUSDC,0.35480000
2638,CRVUSDC,0.43460000


In [49]:
# Check if the request was successful (status code 200)
if response.status_code == 200:

    api_data = response.text
   
    api_data = json.loads(api_data)
        # use this code if the result of the parsed json is in the form {data: {all the response data}}
    # api_data = api_data.get('data', [])

    # Read the JSON data into a Pandas DataFrame
    df = pd.DataFrame(api_data)
    
    # Specify the file path where you want to save the CSV file
    csv_file_path = 'saturday_rate.csv'
    
    # Write the DataFrame to a CSV file
    df.to_csv(csv_file_path, index=False)
    
    print("CSV file saved successfully:", csv_file_path)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

CSV file saved successfully: saturday_rate.csv


In [18]:
# creating a filename variable for compressed file to be uploaded to S3
file_name = csv_file_path + '.gz'
print(file_name)

saturday_rate.csv.gz


In [19]:
df.to_csv(file_name, index=False, compression='gzip')

In [20]:
# df to zip

df.to_csv(file_name, index=False, compression='gzip')

compressed_csv = file_name


In [22]:
with open(compressed_csv, 'rb') as file:
    # Upload the file object to S3 bucket
    s3.upload_fileobj(file, 'emil-coinbase-bucket', compressed_csv)

In [88]:
# Check if the file has been successfully uploaded
if s3.head_object(Bucket='emil-coinbase-bucket', Key=compressed_csv):
    print("CSV file uploaded to S3 bucket successfully")
else:
    print("Upload not successful")

CSV file uploaded to S3 bucket successfully


In [23]:
    # create table for local db
table_name = 'currency_saturday'
rates_table = f'''CREATE TABLE IF NOT EXISTS {table_name} (ID VARCHAR(3) PRIMARY KEY, Name VARCHAR(65), min_size numeric(10, 8) ) '''
connection.cur_pg.execute(rates_table)
connection.conn_pg.commit()



In [26]:
# INSERT DATA TO LOCAL DB
try:
    # size of each batch
    batch_size = 1000

    # Insert DataFrame into database in batches
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i + batch_size]
        values = [tuple(row) for row in batch_df.values]

        placeholders = ','.join(['%s'] * len(df.columns))
        try:
            insert_query = f"INSERT INTO {table_name} (ID, Name, min_size) VALUES ({placeholders})"
            connection.cur_pg.executemany(insert_query, values)
            connection.conn_pg.commit()

            print(insert_query)
        except Exception as e:
            # Rollback the transaction
            connection.conn_pg.rollback()

            # If the insert fails due to a unique constraint violation, perform an update instead
            if 'duplicate key' in str(e):
                for value in values:
                    update_query = "UPDATE currency_rates SET Name = %s, min_size = %s WHERE ID = %s"
                    connection.cur_pg.execute(update_query, (value[1], value[2], value[0]))
                    connection.conn_pg.commit()  # Commit each update
            else:
                raise e

except Exception as e:
    print("An error occurred during batch data insertion:", e)
    connection.conn_pg.rollback()  # Rollback the transaction in case of an error



In [139]:
# INSERT INTO users (id, rate)
# VALUES (1, 0)
# ON CONFLICT (id) DO UPDATE
# SET min_value = %s;


In [136]:
# Upload data to local postgres db 
    # create table
table_name = 'currency_rates'
rates_table = f'''CREATE TABLE IF NOT EXISTS {table_name} (ID VARCHAR(3), Name VARCHAR(65), min_size numeric(10, 8) ) '''
cur_pg.execute(rates_table)
conn_pg.commit()
    # copy csv


# insert_from_local = f"""COPY currency_rates
# FROM '{csv_file_path}'
# DELIMITER ','
# CSV
# HEADER;
#  """

insert_from_local= f"""copy currency_rates FROM '{csv_file_path}' CSV HEADER DELIMITER ','; """


# insert_from_local = f"""COPY currency_rates
# FROM '{csv_file_path}'
# IGNOREHEADER 1
# FORMAT AS csv
# DELIMITER ','; """

cur_pg.execute(insert_from_local)
conn_pg.commit()

conn_pg.close()

UndefinedFile: could not open file "./scratch_csv.csv" for reading: No such file or directory
HINT:  COPY FROM instructs the PostgreSQL server process to read a file. You may want a client-side facility such as psql's \copy.


In [122]:
#  delete this block
connection.cur_redshift.execute('ROLLBACK')

In [123]:
    # create table REDSHIFT


try:
    table_name = 'currency_rates'
    rates_table = f'''CREATE TABLE IF NOT EXISTS {table_name} (
                        ID VARCHAR(3) PRIMARY KEY, 
                        Name VARCHAR(65), 
                        min_size numeric(10, 8)
                    )'''
    
  
    connection.cur_redshift.execute(rates_table)
    connection.conn_redshift.commit()
    print("Table created successfully.")

except Exception as e:
    connection.conn_redshift.rollback()
    print(f"Error creating table: {e}")

finally:
    connection.cur_redshift.close()


In [125]:
# upload data to Redshift
    # create table
table_name = 'currency_rates'
rates_table = f'''CREATE TABLE IF NOT EXISTS {table_name} (ID VARCHAR(3) PRIMARY KEY, Name VARCHAR(65), min_size numeric(10, 8) ) '''
connection.cur_redshift.execute(rates_table)
connection.conn_redshift.commit()
    # copy csv


try:
    # Define the size of each batch
    batch_size = 1000

    # Insert DataFrame into Redshift database in batches
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        values = [tuple(row) for row in batch_df.values]

        placeholders = ','.join(['%s'] * len(df.columns))
        # # column_names = ','.join(df.columns)
        # column_names = ', '.join([f"{re.sub('[^a-zA-Z]+', '', col.replace(' ', '_'))}" for col in df.columns])
        insert_query = f"INSERT INTO currency_rates (ID, Name, min_size) VALUES ({placeholders})"
        print(insert_query)
        connection.cur_redshift.executemany(insert_query, values)
        connection.conn_redshift.commit()

except Exception as e:
    print("An error occurred during batch data insertion:", e)
    connection.conn_redshift.rollback()  # Rollback the transaction in case of an error

aws_access_key_id = connection.aws_access_key_id
aws_secret_access_key = connection.aws_secret_access_key

# insert_from_s3 = f"""COPY currency_rates
# FROM 's3://emil-coinbase-bucket/{csv_file_path}'
# CREDENTIALS 'aws_access_key_id={aws_access_key_id};aws_secret_access_key={aws_secret_access_key}'
# IGNOREHEADER 1
# FORMAT AS csv
# gzip
# DELIMITER ','; """

# insert_from_s3 = f"""COPY currency_rates
# FROM './{csv_file_path}'
# CREDENTIALS 'admin;Hello123'
# IGNOREHEADER 1
# FORMAT AS csv
# DELIMITER ','; """

# connection.cur_redshift.execute(insert_from_s3)
# connection.conn_redshift.commit()

# conn_redshift.close()








INSERT INTO currency_rates (ID, Name, min_size) VALUES (%s,%s,%s)


In [63]:
conn_redshift = connection.conn_redshift
cur_redshift = connection.cur_redshift
print(conn_redshift)

<connection object at 0x000002543DE7EA68; dsn: 'user=admin password=xxx dbname=dev host=coinbase-cluster.ctpfzvbnyqdb.us-east-1.redshift.amazonaws.com port=5439', closed: 0>


In [89]:
cur_redshift.execute('ROLLBACK')

In [42]:


)

None
